<a href="https://colab.research.google.com/github/hirokiOS/SentimentAnalysisWithDownloadedDataSource/blob/main/1_original_accern_parse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境 セットアップ

デフォルトではGoogle colabノートブックを開く度に毎回のpip install実行が必要になります。pipでinstallした開発環境の永続化にはGoogle driveへのPip installが必要です. 以下など参照してください。
https://ggcs.io/2020/06/22/google-colab-pip-install/

In [1]:
!pip install pandas tqdm 
!pip install datasets pyarrow       # convert df to be readable for Transformer classifier
!pip install accern-data

In [ ]:
import pyarrow.dataset as ds
import pyarrow as pa
from datasets import Dataset
import pickle
import json
import pandas as pd
from tqdm import tqdm
# from function import ConvertDfToHFData

## load custom modules from raw python file on github

In [ ]:
import requests
# Save datagenerators as file to colab working directory
# If you are using GitHub, make sure you get the "Raw" version of the code
giturl1 = 'https://raw.githubusercontent.com/hirokiOS/SentimentAnalysisWithDownloadedDataSource/main/acc_function/ACCDFConcatenator.py'
giturl2 = 'https://raw.githubusercontent.com/hirokiOS/SentimentAnalysisWithDownloadedDataSource/main/acc_function/ConvertDfToHFData.py'

giturls = [giturl1, giturl2]

for giturl in giturls:
    r = requests.get(giturl)
    
    filename = giturl.split("/")[-1]
    # make sure your filename is the same as how you want to import 
    with open(filename, 'w') as f:
        f.write(r.text)

# now we can import
from ACCDFConcatenator import DFConcatenator
from ConvertDfToHFData import Convert4SentimentAnalysis

In [ ]:
# see if module loaded
print(Convert4SentimentAnalysis)
print(DFConcatenator)

# Data under accern-raw-json or Accern's API provided

以下のコードを実行するにはログインしてGoogle Driveを紐付ける必要があります。APIや出力の保存先として用います。 \
Prototypingでは有効ですが、内製化する際は自社のサーバ上で構築する必要があります。 \
Local で実行する場合は、Colab_pathに自身の環境のpathを指定してください。

In [ ]:
# if you are running on Google Colab, run the following and generate path to Colaboratory
from google.colab import drive
drive.mount("drive")
colab_path = "drive/MyDrive/Colaboratory/" # Generate Colaboratory folder under you google drive beforehand

In [ ]:
# using google drive, specify the data path. 
!mkdir -p drive/MyDrive/Colaboratory/accern-raw-json # put raw text starting by "{"start_harvested_at":"2023-10-02T13:37:18.544Z","end_harvested_at"..." , which is of accern data
!mkdir -p drive/MyDrive/Colaboratory/data

### 次のコードではdfのファイルソースを選択します。
1. Accern APIの地のテキストをローカルに保存してから読み込み。Google Colab上で./accern-raw-json下に保存したテキストファイルを配置してください。
2. Accern APIのURLから地のテキストとして読み込み
3. (*推奨)Accern提供のAPIを利用して、指定した機関でDataframeとして保存. URL, tokenを設定する必要があり。 

***2. 3. については、適切なWeb sourceをVendorか著者に確認してください。IPのためここでは公開していません。***

In [ ]:
DFCASE = 3

if DFCASE == 3:
    accern_api_url = '< Vender provided URL should come here >'
    accern_token_url = '< Vendor provided token should come here >'

In [ ]:
import urllib.request
# if you save the content to Google drive
SAVEAPICONTENT = False 

if DFCASE == 1:
    # if you already have json file downloaded
    with open('./accern-raw-json/raw-json.json') as f:
        data = json.load(f)

elif DFCASE == 2:
    # download json file from open API source by accern
    url ='< Ask the vendor or author for the web source >'
    
    # if save or directly convert
    if SAVEAPICONTENT == True:
        with urllib.request.urlopen(url) as u:
            with open(colab_path + "/accern-raw-json/raw-json_test.json", 'bw') as o:
                text = u.read()
                o.write(text)
    
        with open(colab_path + "/accern-raw-json/raw-json_test.json") as f:
            data = json.load(f) 

    else: # SAVEAPICONTENT == False 
        with urllib.request.urlopen(url) as u:
            text = u.read()
            data = json.loads(text.decode('utf-8'))
         

elif DFCASE ==3:
    # should contain API. This feature to be implemented in file 2_ 
    df = DFConcatenator(     url=accern_api_url,
                        token=accern_token_url, 
                        start_date="2022-05-01", end_date="2022-05-02", 
                        output_pattern="oct31", output_path="./accern_raw_json/",
                        mode = "json", split_dates=False)


# if fetched via raw api text or test json file, execute the following
if DFCASE==1 or DFCASE==2:
    # Concatenate dataframe
    df = pd.DataFrame()
    df = pd.Series(data['signals'][0]).to_frame().transpose()
    for idx, item in tqdm(enumerate(data['signals'][1:])):
        # dict = data['signals'][0]
        #    df_tmp = pd.Series(data = item)
        df_tmp = pd.Series(item).to_frame().transpose()
        df = pd.concat([df, df_tmp], axis=0, ignore_index=True)

In [ ]:
df.head()

# DF datasetを次の情報を保持するように変換します。 datasets.arrow_dataset.Dataset 型として保存します。
train_dataset.features \
{'sentence': Value(dtype='string', id=None), \
 'label': ClassLabel(names=['positive', 'negative', 'neutral'], id=None), \
 'user_id': Value(dtype='int64', id=None), \
 'datetime': Value(dtype='string', id=None)}

In [ ]:
# hg_dataset = acc_function.Convert4SentimentAnalysis(df, threshold = 30, target_label = 'event_sentiment') # if you downloaded parser for local environment
hg_dataset = Convert4SentimentAnalysis(df, threshold = 30, target_label = 'event_sentiment')


In [ ]:
# show some exampel sentences
hg_dataset['sentence'][:3]

In [ ]:
type(hg_dataset)

# Google drive に接続していれば、Driveに以下でオリジナルのdfおよびHg_dfを保存できます。

### Save original df data as csv for future reference

In [ ]:
df.to_csv(colab_path + '/data/accern_results.csv')

### Save converted Accern data as Pickle file


In [ ]:
with open(colab_path + '/data/accern_dataset.pkl', 'wb') as f:
    pickle.dump( hg_dataset, f )